# Auto Scraper


In [1]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import statistics
import undetected_chromedriver as uc

In [424]:
time.sleep(5)

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.')))).click()


In [425]:
elem_up = driver.find_element(By.TAG_NAME,value="body")

no_of_pagedowns = 30

while no_of_pagedowns:
    elem_up.send_keys(Keys.PAGE_UP+Keys.COMMAND)
    time.sleep(0.25)
    no_of_pagedowns-=1

In [436]:
resultSet = driver.find_element(By.XPATH,"//ul[@class='sui-MoleculePagination']")




In [437]:
print(type(driver))
print(type(resultSet))

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>
<class 'selenium.webdriver.remote.webelement.WebElement'>


In [447]:
list_pages=[]
all_li = resultSet.find_elements(By.TAG_NAME,"li")
for li in all_li:
    text = li.text
    try:
        text=int(text)
        list_pages.append(text)
    except:
        pass
    


max_value=max(list_pages)
    


In [ ]:
for i in range(max_value-1):
        next_page()
        time.sleep(3)
        scroll_up()
        time.sleep(3)
        viviendas=get_html()
        lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion= reset_lists()
        df_loop=get_data(viviendas,lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion)
        df = pd.concat(df,df_loop)
        
        
        
        

In [175]:
barrio = 'San Esteban '
ciudad = 'León'

In [245]:
def initiate (enlace=0):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = uc.Chrome()
    driver.implicitly_wait(5)
    
    driver.get('https://www.fotocasa.es/es/')
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
    time.sleep(3)
    return driver
    

In [289]:
driver= initiate()

In [290]:
def buscar_distrito(distrito, driver):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
    time.sleep(3)
    
    buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
    buscar.send_keys(distrito)
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()

In [291]:
buscar_distrito('san esteban leon',driver)

In [249]:
def scroll_down(driver):  
    elem = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 25
    time.sleep(3)
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
        
    time.sleep(3)
    element = driver.find_element(By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.'))
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()   

In [250]:
scroll_down(driver)

In [292]:
def get_html(driver):
    html_txt=driver.page_source
    soup=BeautifulSoup(html_txt)
    viviendas= soup.find('section',class_='re-SearchResult')
    return viviendas

In [304]:
viviendas=get_html(driver)

In [305]:
def get_data(viviendas):
    try:
        vivienda_str_list=[]

        for vivienda in viviendas:     
            vivienda_str_len=len(str(vivienda))
            vivienda_str_list.append(vivienda_str_len)

        mean_len=statistics.mean(vivienda_str_list)*0.90
        print('Mean length value of the html:',mean_len)
        
        precios = get_precios(viviendas, mean_len)
        print(len(precios))
        titulos = get_titulos(viviendas, mean_len)
        print(len(titulos))

        habitaciones = get_habitaciones(viviendas, mean_len)
        baños = get_baños(viviendas, mean_len)
        superficie = get_superficie(viviendas, mean_len)
        ascensor = get_ascensor (viviendas, mean_len)
        planta = get_planta (viviendas, mean_len)
        link = get_link (viviendas, mean_len)
        fechas = get_publicacion(viviendas, mean_len)
        df=pd.DataFrame(list(zip(titulos,precios,habitaciones,planta,superficie,baños,ascensor,fechas ,link))).drop_duplicates()
        df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})
        barrios=['San Esteban','Palomera']
        ciudades=['León']
        df['Barrio']=barrios[0]
        df['Ciudad']=ciudades[0]
    except:
        df=pd.DataFrame([])
    return df
    
    
def get_precios(viviendas, mean_len):   
    lista_precios=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass

        else:
            precio=vivienda.find('span',class_='re-CardPrice').getText()
            lista_precios.append(precio)
    
    print(len(lista_precios))
    return lista_precios


def get_titulos(viviendas, mean_len):
    lista_titulos=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else: 
            titulos=vivienda.find('span',class_='re-CardTitle').getText()
            lista_titulos.append(titulos)
    print(len(lista_titulos))
    return lista_titulos

def get_habitaciones(viviendas, mean_len):
    lista_habitaciones=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        hab_pos=re.search(r'habs.',vivienda_str)
        hab_pos_indi=re.search(r'hab.',vivienda_str)

        if len(str(vivienda))<mean_len:
            pass

        elif hab_pos:
            hab_pos_text=hab_pos.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)
        else:
            hab_pos_text=hab_pos_indi.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)       

    print(len(lista_habitaciones))
    return lista_habitaciones

def get_baños(viviendas, mean_len):
    lista_baños=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)


        if len(str(vivienda))<mean_len:
            pass

        else:

            baño_pos=re.search(r'(baño|Baño)',vivienda_str)
            baño_pos_text=baño_pos.start()
            b=vivienda_str[baño_pos_text-2:baño_pos_text-1]
            lista_baños.append(b)
 
    print(len(lista_baños))
    return lista_baños
    
def get_superficie (viviendas, mean_len):
    lista_superficie=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        if len(str(vivienda))<mean_len:
            pass
        else:
            superficie_pos=re.search(r'([0-9] [m])',vivienda_str)
            superficie_pos_text=superficie_pos.start()
            s=vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

            if 'm' not in s:
                lista_superficie.append(None)
            else:
                lista_superficie.append(re.findall(r'\d+',s)[0])

    print(len(lista_superficie))
    return lista_superficie

def get_ascensor (viviendas, mean_len):
    lista_ascensor = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        keyword='ascensor'

        if len(str(vivienda))<mean_len:
            pass

        elif keyword in vivienda_str:
            lista_ascensor.append('SI')

        else:
            lista_ascensor.append('NO')
    print(len(lista_ascensor))
    return lista_ascensor

def get_planta (viviendas, mean_len):
    lista_planta = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        planta='Planta'
        planta_baja='planta'

        if len(str(vivienda))<mean_len:
            pass

        elif planta in vivienda_str:
            planta_pos=re.search(r'(Planta)',vivienda_str)
            planta_pos_text=planta_pos.start()
            p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
            lista_planta.append(p)        

        elif planta_baja in vivienda_str:
            planta_pos_baja=re.search(r'(planta)',vivienda_str)
            planta_pos_text_baja=planta_pos_baja.start()
            lista_planta.append(0)        

        else:
            lista_planta.append(None)
        
    print(len(lista_planta))
    return lista_planta
    
def get_link(viviendas, mean_len):
    lista_links = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else:
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista_links.append(fotocasa+href)
                
    print(len(lista_links))
    return lista_links

def get_publicacion (viviendas, mean_len):
    lista_fechas = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass

        else:
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista_fechas.append(fecha_int)
            
    print(len(lista_fechas))
    return lista_fechas


In [312]:
def get_data(viviendas):
    vivienda_str_list=[]

    for vivienda in viviendas:     
        vivienda_str_len=len(str(vivienda))
        vivienda_str_list.append(vivienda_str_len)

    mean_len=statistics.mean(vivienda_str_list)
    print('Mean length value of the html:',mean_len)

    precios = get_precios(viviendas, mean_len)
    titulos = get_titulos(viviendas, mean_len)
    habitaciones = get_habitaciones(viviendas, mean_len)
    baños = get_baños(viviendas, mean_len)
    superficie = get_superficie(viviendas, mean_len)
    ascensor = get_ascensor (viviendas, mean_len)
    planta = get_planta (viviendas, mean_len)
    link = get_link (viviendas, mean_len)
    fechas = get_publicacion(viviendas, mean_len)
    df=pd.DataFrame(list(zip(titulos,precios,habitaciones,planta,superficie,baños,ascensor,fechas ,link))).drop_duplicates()
    df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})
    barrios=['San Esteban','Palomera']
    ciudades=['León']
    df['Barrio']=barrios[0]
    df['Ciudad']=ciudades[0]
    return df
    
    
def get_precios(viviendas, mean_len):   
    lista_precios=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass

        else:
            precio=vivienda.find('span',class_='re-CardPrice').getText()
            lista_precios.append(precio)
            
    
    print(len(lista_precios))
    return lista_precios


def get_titulos(viviendas, mean_len):
    lista_titulos=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else: 
            titulos=vivienda.find('span',class_='re-CardTitle').getText()
            lista_titulos.append(titulos)
    print(len(lista_titulos))
    return lista_titulos

def get_habitaciones(viviendas, mean_len):
    lista_habitaciones=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        hab_pos=re.search(r'habs.',vivienda_str)
        hab_pos_indi=re.search(r'hab.',vivienda_str)

        if len(str(vivienda))<mean_len:
            pass

        elif hab_pos:
            hab_pos_text=hab_pos.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)
        else:
            hab_pos_text=hab_pos_indi.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)       

    print(len(lista_habitaciones))
    return lista_habitaciones

def get_baños(viviendas, mean_len):
    lista_baños=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)


        if len(str(vivienda))<mean_len:
            pass

        else:

            baño_pos=re.search(r'(baño|Baño)',vivienda_str)
            baño_pos_text=baño_pos.start()
            b=vivienda_str[baño_pos_text-2:baño_pos_text-1]
            lista_baños.append(b)
 
    print(len(lista_baños))
    return lista_baños
    
def get_superficie (viviendas, mean_len):
    lista_superficie=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        if len(str(vivienda))<mean_len:
            pass
        else:
            superficie_pos=re.search(r'([0-9] [m])',vivienda_str)
            superficie_pos_text=superficie_pos.start()
            s=vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

            if 'm' not in s:
                lista_superficie.append(None)
            else:
                lista_superficie.append(re.findall(r'\d+',s)[0])

    print(len(lista_superficie))
    return lista_superficie

def get_ascensor (viviendas, mean_len):
    lista_ascensor = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        keyword='ascensor'

        if len(str(vivienda))<mean_len:
            pass

        elif keyword in vivienda_str:
            lista_ascensor.append('SI')

        else:
            lista_ascensor.append('NO')
    print(len(lista_ascensor))
    return lista_ascensor

def get_planta (viviendas, mean_len):
    lista_planta = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        planta='Planta'
        planta_baja='planta'

        if len(str(vivienda))<mean_len:
            pass

        elif planta in vivienda_str:
            planta_pos=re.search(r'(Planta)',vivienda_str)
            planta_pos_text=planta_pos.start()
            p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
            lista_planta.append(p)        

        elif planta_baja in vivienda_str:
            planta_pos_baja=re.search(r'(planta)',vivienda_str)
            planta_pos_text_baja=planta_pos_baja.start()
            lista_planta.append(0)        

        else:
            lista_planta.append(None)
        
    print(len(lista_planta))
    return lista_planta
    
def get_link(viviendas, mean_len):
    lista_links = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else:
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista_links.append(fotocasa+href)
                
    print(len(lista_links))
    return lista_links

def get_publicacion (viviendas, mean_len):
    lista_fechas = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass

        else:
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista_fechas.append(fecha_int)
            
    print(len(lista_fechas))
    return lista_fechas


In [307]:
df=get_data(viviendas)


Mean length value of the html: 5808.872727272727
adios
adios
adios
adios
adios
adios
adios
hola
hola
hola
adios
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
adios
adios
hola
adios
hola
hola
adios
21
21
21
21
21
21
21
21
21


,Título,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
0,"Piso en Calle del Padre García Villada, San Es...",26.998 €,3,2,51,1,SI,35,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
1,"Dúplex con terraza en Calle Peñalba, San Esteban",189.000 €,3,2,103,3,SI,27,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
2,"Piso con terraza en Calle Altos del Duero, San...",109.000 €,3,1,95,2,SI,223,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
3,"Piso con terraza en Padre Isla, San Esteban",115.000 €,3,3,83,1,SI,36,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
4,"Piso con terraza en Alvaro López Núñez, San Es...",139.000 €,4,2,120,2,SI,19,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
5,"Dúplex con terraza en Marqués de Fontiyuelo, S...",232.000 €,4,5,143,2,SI,21,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
6,"Piso con ascensor en Eladio Tejedor, San Esteban",89.000 €,2,4,80,1,SI,33,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
7,"Piso con ascensor en Mariano Andrés, San Esteban",110.000 €,3,4,90,1,SI,57,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
8,Piso con ascensor en San Esteban,95.000 €,3,2,104,1,SI,76,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
9,"Piso con terraza en San Juan de Prado, San Est...",133.000 €,3,1,110,2,SI,76,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León


In [226]:
def get_max_value(driver):
    resultSet = driver.find_element(By.XPATH,"//ul[@class='sui-MoleculePagination']")
    list_pages=[]
    all_li = resultSet.find_elements(By.TAG_NAME,"li")
    for li in all_li:
        text = li.text
        try:
            text=int(text)
            list_pages.append(text)
        except:
            pass
        
    max_value=max(list_pages)
    return max_value

In [227]:
max_value=get_max_value(driver)

In [238]:
def next_page(driver):
    time.sleep(3)
    buttons = driver.find_element(By.CSS_SELECTOR,'.sui-AtomButton.sui-AtomButton--primary.sui-AtomButton--outline.sui-AtomButton--center.sui-AtomButton--small.sui-AtomButton--link.sui-AtomButton--empty.sui-AtomButton--rounded')
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons[1])).click()
    time.sleep(7)

In [253]:
def next_page(driver):
    time.sleep(3)
    buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
    time.sleep(7)

In [243]:
def next_page_first(driver):
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.')))).click()
    time.sleep(7)

In [255]:
next_page(driver)

In [190]:
scroll_down(driver)

In [191]:
viviendas=get_html(driver)

In [193]:
n_df=get_data(viviendas)

Mean length value of the html: 5280.387878787878
30
30
30
30
30
30
30
30
30


In [195]:
df = pd.concat([df,n_df],axis=0)

In [197]:
df.shape

(60, 11)

In [317]:
def batch():
    driver = initiate()
    buscar_distrito('san esteban leon',driver)
    scroll_down(driver)
    viviendas = get_html(driver)
    data=get_data(viviendas)
    max_value=get_max_value(driver)
    for i in range(max_value-1):
        next_page(driver)
        scroll_down(driver)
        viviendas=get_html(driver)
        n_df=get_data(viviendas)
        print(n_df.shape)
        data = pd.concat([data,n_df],axis=0)
        print(df.shape)


In [318]:
if __name__ == '__main__':
    batch()
    print(df.shape)


Mean length value of the html: 7384.848484848485


AttributeError: 'NoneType' object has no attribute 'getText'

In [263]:
get_data

""


In [316]:
df.shape

(21, 11)

## Batch Operation

In [3]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import statistics
import undetected_chromedriver as uc


def initiate (enlace=0):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = uc.Chrome()
    driver.implicitly_wait(5)
    
    driver.get('https://www.fotocasa.es/es/')
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
    time.sleep(3)
    

    
def get_max_value():
    list_pages=[]
    all_li = resultSet.find_elements(By.TAG_NAME,"li")
    for li in all_li:
        text = li.text
        try:
            text=int(text)
            list_pages.append(text)
        except:
            pass
        
    max_value=max(list_pages)
    return max_value
    


def buscar_distrito(distrito):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
    time.sleep(3)
    barrio=ejemplo1
    time.sleep(3)
    
    buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
    buscar.send_keys(barrio)
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()

    
def scroll_down():  
    elem = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 25
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
        
    time.sleep(5)
    element = driver.find_element(By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.'))
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()    


def scroll_up():  
    elem_up = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 30
    while no_of_pagedowns:
        elem_up.send_keys(Keys.PAGE_UP+Keys.COMMAND)
        time.sleep(0.25)
        no_of_pagedowns-=1      
        
        
def get_html():
    html_txt=driver.page_source
    soup=BeautifulSoup(html_txt)
    viviendas= soup.find('section',class_='re-SearchResult')
    return viviendas
    
def reset_lists():
    lista_precios=[]
    lista_titulos=[]
    lista_habitaciones=[]
    lista_baños=[]
    lista_superficie=[]
    lista_planta=[]
    lista_ascensor=[]
    lista_links=[]
    lista_publicacion=[]
    return  lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion
    
def get_data(viviendas,lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion):
    lista_precios=lista_precios
    lista_titulos=lista_titulos
    lista_habitaciones=lista_habitaciones
    lista_baños=lista_baños
    lista_superficie=lista_superficie
    lista_planta=lista_planta
    lista_ascensor=lista_ascensor
    lista_links=lista_links
    lista_publicacion=lista_publicacion
    vivienda_str_list=[]

    for vivienda in viviendas:     
        vivienda_str_len=len(str(vivienda))
        vivienda_str_list.append(vivienda_str_len)
        #print(vivienda_str_len)
        

    mean_len=statistics.mean(vivienda_str_list)*0.73
    #print('this is mean_list:',mean_len)
    
    lista_precios=get_precios(lista_precios)
    lista_titulos=get_precios(lista_titulos)
    lista_habitaciones=get_habitaciones(lista_habitaciones)
    lista_baños=get_baños(lista_baños)
    lista_superficie=get_superficie(lista_superficie)
    lista_planta=get_planta(lista_planta)
    lista_ascensor=get_ascensor(lista_ascensor)
    lista_links=get_links(lista_links)
    lista_publicacion=get_publicacion(lista_publicacion)
    
    df=pd.DataFrame(list(zip(lista_titulos,lista_precios,lista_habitaciones,lista_planta,lista_superficie,lista_baños,lista_ascensor,lista_publicacion ,lista_links))).drop_duplicates()
    df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"}).set_index('Título')
    df['Barrio']=barrios[0]
    df['Ciudad']=ciudades[0]
    print(df)
    return df
    
    
def next_page():
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.')))).click()

    
    
def get_precios(lista_precios):
    lista=lista_precios
    for vivienda in viviendas:     
        try:
            precio=vivienda.find('span',class_='re-CardPrice').getText()
            lista.append(precio)

        except:
            pass
 
    return lista


def get_titulos(lista_titulos):
    lista=lista_titulos
    for vivienda in viviendas:     
        try:
            titulos=vivienda.find('span',class_='re-CardTitle').getText()
            lista.append(titulos)
        except:
            pass
    return lista

    
def get_habitaciones(lista_habitaciones):
    lista=lista_habitaciones
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        hab_pos=re.search(r'habs.',vivienda_str)
        hab_pos_indi=re.search(r'hab.',vivienda_str)

        if len(str(vivienda))<mean_len:
            pass

        elif hab_pos:
            hab_pos_text=hab_pos.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista.append(hab)
        else:
            hab_pos_text=hab_pos_indi.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista.append(hab)       
    
    return lista



def get_baños(lista_baños):
    lista=lista_baños
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        if len(str(vivienda))<mean_len:
            pass
        
        else:

            baño_pos=re.search(r'(baño|Baño)',vivienda_str)
            baño_pos_text=baño_pos.start()
            b=vivienda_str[baño_pos_text-2:baño_pos_text-1]
            lista.append(b)
    return lista


def get_superficie(lista_superficie):
    lista=lista_superficie
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)

        if len(str(vivienda))<mean_len:
            pass

        else:  
            superficie_pos=re.search(r'([0-9] [m2])',vivienda_str)
            superficie_pos_text=superficie_pos.start()
            s=vivienda_str[superficie_pos_text-2:superficie_pos_text+3]
            if 'm' not in s:
                lista.append(0)
            else:
                lista.append(re.findall(r'\d+',s)[0])
    
    return lista


def get_ascensor(lista_ascensor):
    lista=lista_ascensor
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        keyword='ascensor'

        if keyword in vivienda_str:
            lista.append('SI')
        elif len(str(vivienda))<mean_len:
            pass
        else:
            lista.append('NO')

    return lista


def get_planta(lista_planta):
    lista=lista_planta
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        planta='Planta'
        planta_baja='planta'

        if planta in vivienda_str:
            planta_pos=re.search(r'(Planta)',vivienda_str)
            planta_pos_text=planta_pos.start()
            p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
            lista.append(p)      
        elif planta_baja in vivienda_str:
            planta_pos_baja=re.search(r'(planta)',vivienda_str)
            planta_pos_text_baja=planta_pos_baja.start()
            lista.append(0)            
        elif len(str(vivienda))<mean_len:
            pass
        else:
            lista.append(None)
        
    return lista


def get_links(lista_links):
    lista=lista_links
    for vivienda in viviendas:     
        try:
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista.append(fotocasa+href)
        except:
            pass

    return lista



def get_publicacion(lista_publicacion):
    lista=lista_publicacion
    for vivienda in viviendas:     
        try:
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista.append(fecha_int) 
        except:
            pass

    return lista

def run():
    initiate()
    
    time.sleep(5)
    barrios=['san esteban','palomera']
    ciudades=['leon']
    ejemplo1=barrios[0]+'distrito'+ciudades[0]
    
    buscar_distrito(ejemplo1)
    
    time.sleep(5)
    
    scroll_down()
    
    time.sleep(8)
    
    max_value=get_max_value()
    
    viviendas=get_html()
    lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion= reset_lists()
    df=get_data(viviendas,lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion)

    
    for i in range(max_value-1):
        next_page()
        time.sleep(3)
        scroll_up()
        time.sleep(3)
        viviendas=get_html()
        lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion= reset_lists()
        df_loop=get_data(viviendas,lista_precios,lista_titulos,lista_habitaciones,lista_baños, lista_superficie,lista_planta, lista_ascensor, lista_links, lista_publicacion)
        df = pd.concat([df,df_loop],axis=0)
    

        
        
    

    
 


In [4]:
run()

NameError: name 'driver' is not defined

In [617]:
df

,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
Título,,,,,,,,,,
95.000 €,95.000 €,3,2,114,1,NO,35,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
136.000 €,136.000 €,3,None,180,1,NO,37,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
59.000 €,59.000 €,4,None,190,2,NO,22,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
99.000 €,99.000 €,2,3,70,1,SI,0,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
110.000 €,110.000 €,3,4,80,1,SI,21,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
269.000 €,269.000 €,3,None,131,2,SI,39,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
178.000 €,178.000 €,3,6,None,2,SI,0,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
115.000 €,115.000 €,3,3,97,1,SI,59,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon
140.000 €,140.000 €,3,5,108,1,SI,22,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon


In [619]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 95.000 € to 56.000 €
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Precio             226 non-null    object
 1   Habitaciones       226 non-null    object
 2   Planta             189 non-null    object
 3   Superficie         198 non-null    object
 4   Baños              226 non-null    object
 5   Ascensor           226 non-null    object
 6   Fecha Publicación  226 non-null    int64 
 7   Links              226 non-null    object
 8   Barrio             226 non-null    object
 9   Ciudad             226 non-null    object
dtypes: int64(1), object(9)
memory usage: 19.4+ KB


In [620]:
df.describe()

,Fecha Publicación
count,226.000000
mean,37.101770
std,58.680023
min,0.000000
25%,11.000000
50%,16.000000
75%,35.000000
max,367.000000


In [634]:
df['Precio']=df['Precio'].str.replace(r'€', '').str.replace(r'.', '').astype(int)

/var/folders/zk/slrtmhdd7kd3g8n3jb1bd1700000gn/T/ipykernel_12914/1850164513.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio']=df['Precio'].str.replace(r'€', '').str.replace(r'.', '').astype(int)


In [ ]:
df = df.fillna(value=np.nan)


In [637]:
df['Superficie']=df['Superficie'].fillna(value=0).astype(int)

In [638]:
df['Precio/m2']=df['Precio']/df['Superficie']

In [639]:
df['Precio/m2']

Título
95.000 €      833.333333
136.000 €     755.555556
59.000 €      310.526316
99.000 €     1414.285714
110.000 €    1375.000000
269.000 €    2053.435115
178.000 €            inf
115.000 €    1185.567010
140.000 €    1296.296296
125.000 €    1470.588235
145.000 €    1421.568627
27.500 €      539.215686
34.000 €             inf
58.000 €             inf
85.000 €      410.628019
89.000 €     1435.483871
51.000 €      548.387097
99.000 €     1053.191489
58.000 €             inf
79.500 €     1074.324324
125.000 €    1506.024096
89.000 €     1435.483871
49.900 €      674.324324
147.000 €    1256.410256
145.000 €    1098.484848
31.000 €      402.597403
107.000 €    1507.042254
89.000 €     1328.358209
92.000 €     1243.243243
56.000 €      823.529412
90.000 €      439.024390
600.000 €    1109.057301
189.000 €    1410.447761
72.000 €      888.888889
99.000 €     1010.204082
95.000 €      848.214286
215.000 €    1557.971014
65.000 €      714.285714
195.000 €    2166.666667
95.000 €          

In [642]:
duplicate = df[df.duplicated(['Superficie','Habitaciones'])]

In [643]:
duplicate

,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad,Precio/m2
Título,,,,,,,,,,,
34.000 €,34000,3,1,0,1,SI,0,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,inf
58.000 €,58000,3,3,0,1,NO,7,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,inf
89.000 €,89000,2,4,62,1,SI,11,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,1435.483871
49.900 €,49900,3,4,74,1,NO,10,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,674.324324
95.000 €,95000,2,2,0,2,SI,3,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,inf
89.000 €,89000,2,4,0,1,SI,42,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,inf
92.000 €,92000,2,1,0,1,SI,34,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,inf
180.000 €,180000,3,None,102,2,SI,134,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,1764.705882
148.000 €,148000,3,0,100,2,SI,367,https://www.fotocasa.es//es/comprar/vivienda/l...,san esteban distrito leon,leon,1480.000000


## Modelo Precios de Alquileres

1. Entrenar los modelos en base a los precios de los alquileres del barrio
2. Predecir los precios de alquiler aproximados para los pisos en venta
3. Calcular la rentabilidad
4. Calcular precio para la rentabilidad deseada